In [ ]:
# prompt: mount

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Cell 1 ▶ Consolidated imports
import os, math, glob, time
import numpy as np, pandas as pd
from pathlib import Path

import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on", device)


Running on cuda


In [ ]:
# ─── Cell 1: Imports & Dataset ─────────────────────────────────────────────
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

class DateTileDataset(Dataset):
    """
    Each row in file_to_meteo.csv gives:
      - date: "YYYY-MM-DD"
      - file_path: full path to X_YYYY-MM-DD.npy  (shape (N,6,6,8))
      - air_temp_C, dew_point_C, relative_humidity_percent, wind_speed_m_s, precipitation_in
    We load X (tiles×6×6×8), Y (tiles×K) from Y_imf_{date}.npy, and the 5-vector of weather.
    """
    def __init__(self, lookup_csv, x_dir, y_dir):
        self.meta  = pd.read_csv(lookup_csv)
        self.x_dir = x_dir
        self.y_dir = y_dir

    def __len__(self):
        return len(self.meta)

    def __getitem__(self, idx):
        row  = self.meta.iloc[idx]
        date = row['date']
        # load X, Y
        X    = np.load(row['file_path'])                                    # (N,6,6,8)
        Y    = np.load(os.path.join(self.y_dir, f"Y_imf_{date}.npy"))       # (N,K)

        # weather vector
        met  = row[['air_temp_C','dew_point_C',
                    'relative_humidity_percent',
                    'wind_speed_m_s','precipitation_in']].values.astype(np.float32)

        # to torch
        X    = torch.from_numpy(X).permute(0,3,1,2).float()  # (N,8,6,6)
        Y    = torch.from_numpy(Y).float()                   # (N,K)
        met  = torch.from_numpy(met).float()                 # (5,)

        return X, met, Y

In [ ]:
# ─── Cell 2: train/val split & DataLoaders ─────────────────────────────────
from torch.utils.data import DataLoader

# adjust these to your Drive paths:
LOOKUP_CSV = "/content/drive/MyDrive/EarthEngine/file_to_meteo.csv"
X_DIR      = "/content/drive/MyDrive/EarthEngine/X"
Y_DIR      = "/content/drive/MyDrive/EarthEngine/Y_imf"

# load & split by date
lookup = pd.read_csv(LOOKUP_CSV)
val_df = lookup.sample(frac=0.2, random_state=42).reset_index(drop=True)
tr_df  = lookup.drop(val_df.index).reset_index(drop=True)

# Cell 2 ▶ wrap in Dataset, then override .meta
train_ds = DateTileDataset(
    x_dir       = X_DIR,
    y_dir       = Y_DIR,
    lookup_csv  = LOOKUP_CSV   # <— point at your CSV!
)
train_ds.meta = tr_df           # now replace with just the train rows

val_ds = DateTileDataset(
    x_dir       = X_DIR,
    y_dir       = Y_DIR,
    lookup_csv  = LOOKUP_CSV
)
val_ds.meta = val_df            # replace with just the val rows

# DataLoaders not used directly (we iterate per-date inside our loop)
# but you could also flatten every tile into one giant loader:
# train_loader = DataLoader(train_ds, batch_size=1, shuffle=True)


In [ ]:
# Cell 3 ▶ Model, optimizer, AMP, etc.
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler

# ── 1) Define your network class ─────────────────────────────────────────
class PIHPNet_SingleScale(nn.Module):
    def __init__(self, in_gscfm=5, in_idx=3, meteo_in=5, hidden=64, K=3):
        super().__init__()
        self.enc_g = nn.Sequential(
            nn.Conv2d(in_gscfm, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32,   64, 3, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
        )
        self.enc_i = nn.Sequential(
            nn.Conv2d(in_idx, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32,    64, 3, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
        )
        self.metmlp = nn.Sequential(
            nn.Linear(meteo_in, hidden), nn.ReLU(),
            nn.Linear(hidden,    hidden), nn.ReLU(),
        )
        fuse_dim = 64 + 64 + hidden
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(fuse_dim, fuse_dim//2), nn.ReLU(),
                nn.Linear(fuse_dim//2, 1),
            ) for _ in range(K)
        ])

    def forward(self, X, met):
        gscfm = X[:,:5]
        idx   = X[:,5:]
        fg    = self.enc_g(gscfm).view(X.size(0), -1)
        fi    = self.enc_i(idx).view(X.size(0), -1)
        fm    = self.metmlp(met).expand(X.size(0), -1)
        H     = torch.cat([fg, fi, fm], dim=1)
        out   = [head(H) for head in self.heads]
        return torch.cat(out, dim=1)

# ── 2) Instantiate & move to GPU ─────────────────────────────────────────
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model   = PIHPNet_SingleScale().to(device)

# ── 3) Loss, optimizer, scaler ────────────────────────────────────────────
crit    = nn.MSELoss()
opt     = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scaler  = GradScaler() if torch.cuda.is_available() else None
clip_val = 5.0
mini_bs  = 512
epochs   = 10


<ipython-input-11-06b8d261dea5>:50: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler  = GradScaler() if torch.cuda.is_available() else None


In [ ]:
# ─── Cell 4: Training + Validation Loop ────────────────────────────────────
from tqdm import tqdm

for ep in range(1, epochs+1):
    # — TRAIN —
    model.train()
    running_loss = 0.0
    seen_tiles   = 0

    print(f"\n=== Epoch {ep}/{epochs} TRAIN ===")
    for _, row in tr_df.sample(frac=1).iterrows():
        date   = row['date']
        X_all  = np.load(row['file_path'])
        Y_all  = np.load(f"{Y_DIR}/Y_imf_{date}.npy")
        N      = X_all.shape[0]

        met = torch.tensor([
            row.air_temp_C, row.dew_point_C,
            row.relative_humidity_percent,
            row.wind_speed_m_s, row.precipitation_in
        ], device=device, dtype=torch.float32)

        perm = np.random.permutation(N)
        pbar = tqdm(total=N, desc=date, unit='tiles', leave=False)
        for i in range(0, N, mini_bs):
            idx = perm[i:i+mini_bs]
            Xb  = torch.from_numpy(X_all[idx]).permute(0,3,1,2).to(device)
            Yb  = torch.from_numpy(Y_all[idx]).to(device)
            mb  = met.unsqueeze(0).expand(len(idx), -1)

            opt.zero_grad()
            with autocast(enabled=scaler is not None):
                pred = model(Xb, mb)
                loss = crit(pred, Yb)
            if scaler:
                scaler.scale(loss).backward()
                gnorm = nn.utils.clip_grad_norm_(model.parameters(), clip_val)
                scaler.step(opt)
                scaler.update()
            else:
                loss.backward()
                gnorm = nn.utils.clip_grad_norm_(model.parameters(), clip_val)
                opt.step()

            b = len(idx)
            running_loss += loss.item() * b
            seen_tiles   += b
            pbar.set_postfix(loss=f"{running_loss/seen_tiles:.4f}", gnorm=f"{min(gnorm,clip_val):.1f}")
            pbar.update(b)
        pbar.close()

    print(f"→ TRAIN MSE/tile: {running_loss/seen_tiles:.6f}")

    # — VALIDATE —
    model.eval()
    val_loss  = 0.0
    val_tiles = 0

    print(f"=== Epoch {ep}/{epochs}  VAL ===")
    for _, row in val_df.iterrows():
        date   = row['date']
        X_all  = np.load(row['file_path'])
        Y_all  = np.load(f"{Y_DIR}/Y_imf_{date}.npy")
        N      = X_all.shape[0]

        met = torch.tensor([
            row.air_temp_C, row.dew_point_C,
            row.relative_humidity_percent,
            row.wind_speed_m_s, row.precipitation_in
        ], device=device, dtype=torch.float32)

        pbar = tqdm(total=N, desc=date, unit='tiles', leave=False)
        with torch.no_grad():
            for i in range(0, N, mini_bs):
                idx = range(i, min(i+mini_bs, N))
                Xb  = torch.from_numpy(X_all[idx]).permute(0,3,1,2).to(device)
                Yb  = torch.from_numpy(Y_all[idx]).to(device)
                mb  = met.unsqueeze(0).expand(len(idx), -1)

                pred = model(Xb, mb)
                loss = crit(pred, Yb)

                b = len(idx)
                val_loss  += loss.item() * b
                val_tiles += b
                pbar.set_postfix(loss=f"{val_loss/val_tiles:.4f}")
                pbar.update(b)
        pbar.close()

    print(f"→  VAL MSE/tile: {val_loss/val_tiles:.6f}")



=== Epoch 1/10 TRAIN ===


2020-01-17:   0%|          | 0/1658958 [00:00<?, ?tiles/s]<ipython-input-12-155e0420ae60>:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=scaler is not None):


→ TRAIN MSE/tile: 164.707179
=== Epoch 1/10  VAL ===


→  VAL MSE/tile: 241.935916

=== Epoch 2/10 TRAIN ===


→ TRAIN MSE/tile: 179.397527
=== Epoch 2/10  VAL ===


...

→ TRAIN MSE/tile: 171.356009
=== Epoch 3/10  VAL ===


→  VAL MSE/tile: 218.715704

=== Epoch 4/10 TRAIN ===


→ TRAIN MSE/tile: 193.143114
=== Epoch 4/10  VAL ===


→  VAL MSE/tile: 216.074303

=== Epoch 5/10 TRAIN ===


ValueError: cannot reshape array of size 167903200 into shape (1659888,6,6,8)